In [1]:
import os
import re
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import transformers
from transformers import PreTrainedTokenizerFast
from transformers import LlamaConfig, LlamaModel, LlamaForCausalLM, TrainingArguments, DataCollatorForLanguageModeling, Trainer, TrainerCallback, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model


e:\repos\LLM_pretrain_and_SFT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

# Pretrain

### 1. Подготовка данных

In [3]:
novels_path ='../RussianNovels/corpus/'

In [4]:
texts = []

for file in os.listdir(novels_path):
    with open(novels_path + file, 'r', encoding='utf-8') as f:
        texts.append(f.read())

len(texts)

108

In [5]:
# Удаление дубликатов
texts = list(set(texts))
len(texts)

107

In [6]:
# Удаление предложений с буквами не из кириллицы
for i, text in enumerate(texts):
    sents_with_latin = re.findall(r'[^.!?]*[a-zA-Z]+[^.!?]*[.!?]', text)
    for sent in sents_with_latin:
        texts[i] = texts[i].replace(sent, '')

In [7]:
# Удаление повторяющейся пунктуации
for i, text in enumerate(texts):
    texts[i] = re.sub(r'([.,!?])\1+', r'\1', text)

In [8]:
# Деление текстов на чанки
texts_split = []

chunk_len = 1000 # Длина чанка (в символах)

for text in texts:
    l = int(np.ceil(len(text)/chunk_len))
    texts_split.append(text[0:chunk_len])
    for i in range(1,l):
        texts_split.append(text[chunk_len*i:chunk_len*i+chunk_len])

len(texts_split)

43066

### 2. Создание токенизатора

In [9]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [10]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [11]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=3000, special_tokens=special_tokens)

In [12]:
tokenizer.train_from_iterator(texts_split, trainer=trainer)

In [13]:
encoding = tokenizer.encode("Ну как, работает?")
print(encoding.tokens)

['ну', 'как', ',', 'рабо', '##та', '##ет', '?']


In [14]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [15]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [16]:
encoding = tokenizer.encode("Ну как, работает?")
print(encoding.tokens)

['[CLS]', 'ну', 'как', ',', 'рабо', '##та', '##ет', '?', '[SEP]']


In [17]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [18]:
tokenizer.decode(encoding.ids)

'ну как, работает?'

In [19]:
tokenizer.save("tokenizer.json")

In [20]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # В качестве альтернативы можно загрузить из файла токенизатора.
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

### 3. Создание Dataset-а

In [21]:
def tokenize_function(examples):
    return wrapped_tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = Dataset.from_dict({"text": texts_split})
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.remove_columns(["text"])

Map: 100%|██████████| 43066/43066 [00:08<00:00, 5117.82 examples/s]


In [22]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=wrapped_tokenizer,
    mlm=False,
)

### 4. Инициализация модели

In [23]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    vocab_size=32000,
    max_position_embeddings=2048,
)

model = LlamaForCausalLM(config)

In [24]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

In [25]:
# Коллбэк для валидации качества генерации
class ValidationCallback(TrainerCallback):
    def __init__(self, prompts, tokenizer, every_n_steps=500):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.every_n_steps = every_n_steps
        
    def on_step_end(self, args, state, control, model, **kwargs):
        if state.global_step % self.every_n_steps == 0 and state.global_step > 0:
            print(f"\n--- Валидация на шаге {state.global_step} ---")
            model.eval()
            with torch.no_grad():
                for i, prompt in enumerate(self.prompts):
                    inputs = self.tokenizer(prompt, return_tensors="pt")
                    
                    device = next(model.parameters()).device
                    inputs = {k: v.to(device) for k, v in inputs.items()}
                    
                    outputs = model.generate(
                        inputs['input_ids'],
                        max_length=len(inputs['input_ids'][0]) + 50,
                        num_return_sequences=1,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=self.tokenizer.eos_token_id,
                        repetition_penalty=1.1
                    )
                    generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                    print(f"{i+1}. Промпт: {prompt}")
                    print(f"   Сгенерировано: {generated_text}")
            print("---")
            model.train()

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_llm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8, # Из-за ограничений собственных ресурсов...
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False,
    weight_decay=0.01,  # Регуляризация
    logging_steps=100,
)


In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    callbacks=[ValidationCallback(test_prompts, wrapped_tokenizer, every_n_steps=1000)]
)


In [28]:
trainer.train()

Step,Training Loss
100,7.481300
200,6.457300
300,6.145700
400,5.887200
500,5.664200
600,5.465300
700,5.294000
800,5.162100
900,5.050700
1000,4.939800



--- Валидация на шаге 1000 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Промпт: Все мысли, которые имеют огромные последствия
   Сгенерировано: все мысли, которые имеют огромные последствиянного духа, как в самом деле, в его разные моктике, сбирным, пронзионом, улег, на все было, кляну и взахнулся, и не по


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


8. Промпт: Всякая мысль, даже самая простая
   Сгенерировано: всякая мысль, даже самая простаяичу в этом деле! о чем бы он ни был, так и думал бы, что он так уж был, как и прежде. но ведь я его не знаю. ну, да и про с вами тоже я все - таки об этом говорю


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


9. Промпт: Война не любезность, а самое гадкое дело
   Сгенерировано: воина не любезность, а самое гадкое делом, которое вы не можете быть из этого положения. она не могла понять и даже теперь. вы знаете, что он знает о неи, как бы с своим чувством сделать это слово, в котором она играла. – но я думаю
10. Промпт: Чтобы жить честно
   Сгенерировано: чтобы жить честнон, я не умею. -- господа, не беспокоитесь! мне надобно, -- сказал растопчин и снова начал рыдать : -- я не могу. мне же тяжело. -- что вы? я знаю, что я
---


TrainOutput(global_step=16152, training_loss=3.768287294037334, metrics={'train_runtime': 27541.8586, 'train_samples_per_second': 4.691, 'train_steps_per_second': 0.586, 'total_flos': 4.325441616062054e+16, 'train_loss': 3.768287294037334, 'epoch': 3.0})

In [29]:
model.eval()
results = []

for i, prompt in enumerate(test_prompts):
    inputs = wrapped_tokenizer(prompt, return_tensors="pt")
    
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=len(inputs['input_ids'][0]) + 100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=wrapped_tokenizer.eos_token_id,
            top_p=0.9,
            repetition_penalty=1.1
        )
    
    generated_text = wrapped_tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append({
        "№": i+1,
        "Промпт": prompt,
        "Сгенерированный текст": generated_text
    })
    
    print(f"/n{i+1}. ПРОМПТ: {prompt}")
    print(f"РЕЗУЛЬТАТ: {generated_text}")
    print("-" * 80, "/n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n1. ПРОМПТ: Все мысли, которые имеют огромные последствия
РЕЗУЛЬТАТ: все мысли, которые имеют огромные последствиям и унылостию. но что это случилось? я не мог понять, как все это было. мне хотелось видеть этот разговор, которого я боялась. -- нет, -- сказала она, -- я ничего не понимаю, потому что мне нужно сказать что - нибудь. -- отчего же вы не знаете, зачем вы это думаете? -- сказал он. -- ах, что вы хотите! -- отвечала она, взяв ее за руку. -- нет, -- сказал он еи,
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n2. ПРОМПТ: Сила войска зависит от его духа
РЕЗУЛЬТАТ: сила воиска зависит от его духа и тому подобное. а вот оно бывает, таковое чувство! -- я вам скажу : не понимаю, как это было так, потому что я была бы счастлива, как и все, что ты думаешь о неи. -- как же? -- да и о чем она спрашивала. -- о, я думала, -- промолвила анна михаиловна, -- но ведь нельзя же мне и говорить. -- ну, вот и все! -- прибавила она, обращая
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

/n7. ПРОМПТ: Нет, жизнь не кончена
РЕЗУЛЬТАТ: нет, жизнь не конченарскими донцами. - вы хотите быть вашим? - спросил самгин. - а если вы не могли сказать мне, я бы сдержался в вашем будущем. - я не понимаю, что вы не можете сделать мне. но я чувствую, что вы все понимаете. я вас не знаю. я не понимаю. - я понимаю. - и - не понимаю. - почему же? - он вдруг задыхался в густои бородке,
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n8. ПРОМПТ: Всякая мысль, даже самая простая
РЕЗУЛЬТАТ: всякая мысль, даже самая простаяная, а потом уж и не покидалась. " я в самом деле, как вы меня сделали!" - подумала она, глядя на него и укоризненно глядя в лицо. - вот вы мне ничего не говорите, - сказала она, - вы не знаете, что я сегодня на вас спрашивала. да, это очень серьезно, если вам хочется сделать, что я вам скажу, что вы мне все # таки очень скучно ; но это не по
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n9. ПРОМПТ: Война не любезность, а самое гадкое дело
РЕЗУЛЬТАТ: воина не любезность, а самое гадкое деломное и с ним, - и он, в сущности, не веря в своеи силе, вроде его влюбленного, но в то же время уверен, что в нем ничего, кроме его, что он в этом - то деле и не умел сделать. " нет, я думаю, не могу, - говорит ему яков, - вот это - чепуха, - и не могу, не для того, чтобы все это были ". но, когда
-------------------------------------------------------------------------------- /n
/n10. ПРОМПТ: Чтобы жить честно
РЕЗУЛЬТАТ: чтобы жить честнон, не желая узнать, как этот батальон, по - видимому, по - новому, подходит к тому месту, где приметы дела, и что у него еще не было, и все это он мог бы быть неловко, а как будто ему неловко было думать о чем - нибудь. но в этои комнате, где он, как обыкновенно, с отвращением вспомнились его слова, они с ним чувствовали
-------------------------------------------------------------------------------- /n


# Post-train SFT

### 1. Подготовка датасета

In [12]:

def format_dialog(example):
    system_msg = example.get('input', '')
    user_msg = example.get('user', '')
    assistant_msg = example.get('assistant', '')
    
    # Формат Qwen chat template
    formatted_text = f"<|im_start|>system\n{system_msg}<|im_end|>\n<|im_start|>user\n{user_msg}<|im_end|>\n<|im_start|>assistant\n{assistant_msg}<|im_end|>"
    
    return {"text": formatted_text}

def prepare_alpaca_dataset(dataset):
    
    formatted_dataset = dataset.map(format_dialog)
    return formatted_dataset

dataset = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
alpaca_dataset = prepare_alpaca_dataset(dataset)


Map: 100%|██████████| 51760/51760 [00:01<00:00, 30985.03 examples/s]


### 2. Дообучение модели

In [6]:
model_name = "Qwen/Qwen2.5-0.5B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [7]:
model.to('cuda')

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    output_dir="./qwen2.5-0.5b-alpaca",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to=None,
    ddp_find_unused_parameters=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=alpaca_dataset,
    peft_config=lora_config,
)

trainer.train()

trainer.save_model()

Truncating train dataset: 100%|██████████| 51760/51760 [00:00<00:00, 235298.71 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,6.274100
20,4.686500
30,3.590500
40,3.033300


KeyboardInterrupt: 

### 3. Генерация ответов

In [20]:
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
  ] 

In [ ]:
for question in questions_rus:
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print('\nВОПРОС:', question)
    print("ОТВЕТ:", response, '\n')